In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Flatten, Resizing, Rescaling
from tensorflow.keras.models import Model

from tensorflow.keras.layers import LayerNormalization
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from tensorflow.keras import backend as K
from tensorflow_addons.layers import GroupNormalization, InstanceNormalization
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer



C:\Users\adeto\AppData\Roaming\Python\Python311\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\adeto\AppData\Roaming\Python\Python311\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file

In [2]:
#pip install clyent==1.2.1 pyyaml==6.0.1


In [3]:
#pip install tensorflow-privacy==0.5.0


In [4]:
# Load CIFAR10 dataset: 60k 32x32 RGB images (50k training set + 10k test set)
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()


In [5]:
# Normalize pixel values
#train_data, test_data = train_data / 255.0, test_data / 255.0

In [6]:
# Normalize the pixel values of the train and test data
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

In [7]:
# ResNet50 expects input size of (224, 224, 3)
resize_and_rescale = tf.keras.Sequential([
    Resizing(224, 224),
    Rescaling(1./255)
])

In [8]:
#10 lAYERS
# This acheives higher accuracy
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

#Next high accuracy
models = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),  # Layer 1: Conv2D
    LayerNormalization(),  # Layer 2: LayerNormalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Layer 3: MaxPooling2D
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),  # Layer 4: Conv2D
    tf.keras.layers.Dropout(0.25),  # Layer 5: Dropout
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),  # Layer 6: Conv2D
    GroupNormalization(),  # Layer 7: GroupNormalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Layer 8: MaxPooling2D
    tf.keras.layers.Flatten(),  # Layer 9: Flatten
    tf.keras.layers.Dense(128, activation='relu'),  # Layer 10: Dense
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer: Dense with softmax for multi-class classification
])


In [9]:
# Define the ResNet50 model
# Resizing and Rescaling layers (considered as a part of preprocessing)
input_tensor = Input(shape=(32, 32, 3))
x = resize_and_rescale(input_tensor)

#ResNet50 has 50 layers
base_model = ResNet50(weights=None, include_top=False, input_tensor=x)

#Flattens the base model
x = Flatten()(base_model.output)
output_layer = Dense(10, activation='softmax')(x)  # CIFAR10 has 10 classes

#model = Model(inputs=input_tensor, outputs=output_layer)


In [10]:
# Model summary to verify the number of layers
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 128)         0         
 g2D)                                                 

In [11]:
# Define parameters for DP-SGD
noise_multiplier = 2.5  # Adjusted for tighter privacy guarantee
l2_norm_clip = 1.0
batch_size = 300 
learning_rate = 0.001
epochs = 400 

In [12]:

#DP - Adam Oprimizer
'''
optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_size,
    learning_rate=learning_rate
)
'''


# Create DP-SGD optimizer model
optimizers = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_size,
    learning_rate=learning_rate
)

#l2_norm_clip: This parameter sets a threshold for clipping the L2 norm of gradients, which prevents any single 
#data point from having a disproportionate impact on the computation of gradients, thereby safeguarding individual 
#data point privacy.

#noise_multiplier: This determines the amount of random noise added to the gradients during training. The noise helps 
#mask the contribution of individual data points, which is central to achieving differential privacy.

#num_microbatches: This parameter controls the subdivision of a batch into smaller units, or microbatches. Processing 
#these microbatches separately and then aggregating their gradients ensures that the influence of any single data point 
#is limited.

#learning_rate: While not directly related to privacy, the learning rate can impact the convergence of the training 
#process, especially when combined with the other DP parameters.


In [13]:
#loss
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [14]:
# Define RMSE as a custom metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [15]:
#define model parameters
model.compile(optimizer=optimizers, loss=loss, metrics=['accuracy', rmse])

In [16]:
# Train the model on 50k training images and validate on 10k test images
# Train the model with differential privacy
history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data, test_labels))

Epoch 1/400


167/167 [==============================] - 147s 874ms/step - loss: 2.3034 - accuracy: 0.1020 - rmse: 5.2537 - val_loss: 2.2983 - val_accuracy: 0.1137 - val_rmse: 5.2471
Epoch 2/400
167/167 [==============================] - 169s 1s/step - loss: 2.2968 - accuracy: 0.1162 - rmse: 5.2530 - val_loss: 2.2937 - val_accuracy: 0.1349 - val_rmse: 5.2471
Epoch 3/400
167/167 [==============================] - 168s 1s/step - loss: 2.2930 - accuracy: 0.1287 - rmse: 5.2524 - val_loss: 2.2895 - val_accuracy: 0.1658 - val_rmse: 5.2471
Epoch 4/400
167/167 [==============================] - 187s 1s/step - loss: 2.2894 - accuracy: 0.1367 - rmse: 5.2525 - val_loss: 2.2850 - val_accuracy: 0.1840 - val_rmse: 5.2471
Epoch 5/400
167/167 [==============================] - 190s 1s/step - loss: 2.2848 - accuracy: 0.1490 - rmse: 5.2518 - val_loss: 2.2800 - val_accuracy: 0.1962 - val_rmse: 5.2471
Epoch 6/400
167/167 [==============================] - 189s 1s/step - loss: 2.2793 - accuracy: 0.1571 - r

In [17]:
# After training, multiply accuracy by 100 to convert to percentage
final_accuracy = history.history['accuracy'][-1] * 100
final_val_accuracy = history.history['val_accuracy'][-1] * 100


In [18]:
print(f"Final training accuracy: {final_accuracy}%")
print(f"Final validation accuracy: {final_val_accuracy}%")

Final training accuracy: 68.23199987411499%
Final validation accuracy: 65.21000266075134%


In [19]:
# Compute the privacy budget expended during training
epsilon, delta = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    len(train_data), batch_size, noise_multiplier, epochs, delta=1e-5)
print(f"Trained with DP-SGD with ε = {epsilon:.2f} and δ = {delta}")

DP-SGD with sampling rate = 0.6% and noise_multiplier = 2.5 iterated over 66667 steps satisfies differential privacy with eps = 3.32 and delta = 1e-05.
The optimal RDP order is 8.0.
Trained with DP-SGD with ε = 3.32 and δ = 8.0


In [ ]:
#def compute_dp_adam_privacy(train_data_length, batch_size, noise_multiplier, epochs, delta=1e-5):
    # Placeholder for privacy calculation logic
    # The actual implementation would depend on the differential privacy library being used.
    # For example, TensorFl   ow Privacy provides tools to compute the privacy budget (epsilon).

    # The function below is conceptual and does not represent actual code from TensorFlow Privacy or similar libraries.
    # You will need to replace this with the appropriate function call from the differential privacy library you are using.
    #epsilon = some_dp_library.compute_epsilon(train_data_length, batch_size, noise_multiplier, epochs, delta)

    #return epsilon, delta

In [ ]:
# Compute the privacy budget expended during training
#epsilon, delta = compute_dp_adam_privacy(len(train_data), batch_size, noise_multiplier, epochs)
#print("Epsilon:", epsilon, "Delta:", delta)